# Yoda: Python API danger zone

**NOTE**: The python API gives you a lot of power and you can do a lot of misschief.
Here we show some of that and the implications

In [1]:
import getpass
import os
import json

from irods.session import iRODSSession

## Connecting to iRODS

### Login to Yoda instances
To connect to Yoda you will need an environment file. These files can be found [here (Step 2. Configuring iCommands)](https://www.uu.nl/en/research/yoda/guide-to-yoda/i-am-using-yoda/using-icommands-for-large-datasets). Please adjust the username in the file for your Yoda instance and store that file in your home-directory `~/.irods/irods-environment.json`.

You will also need a [Data Access Password](https://www.uu.nl/en/research/yoda/using-data-access-passwords). 

In [7]:
passwd = getpass.getpass("Yoda data access password")

Yoda data access password········


In [8]:
with open(os.path.expanduser("~/.irods/irods_environment.json"), "r") as f:
    ienv = json.load(f)
session = iRODSSession(**ienv, password=passwd)

## Changing Yoda metadata for archiving
Prerequisites:

1) Create a collection with one or two files
2) Fill in the metadata form in yoda web-portal
3) submite this collection to Vault


In the previous tutorials we created own metadata tags. Now let's have a look at a collection which we already annotated with metadata in the Yoda web interface:

In [9]:
homeCollPath = '/' + session.zone + '/home/research-test-christine'
collPath = "blabla"
coll = session.collections.get(homeCollPath +'/'+ collPath)
print(coll)

<iRODSCollection 21260094 b'blabla'>


Now we can access the single metadata items and have a look at them:

In [ ]:
for item in coll.metadata.items():
    print(item.name, item.value, item.units)

We see that there is a lot of metadata. Among the items you can find your metadata that you addded in the metadata.json, which is shown in the Yoda web front end in the metadata mask.

What happens if we change that metadata? Let's change one of the Yoda metadata items and see what happens in the web interface.
We will remove **Family_Name Staiger usr_6_s**.

In [ ]:
coll.metadata.remove("Family_Name", "Staiger", "usr_6_s")

In [ ]:
for item in coll.metadata.items():
    print(item.name, item.value, item.units)

We see, that the metadata in the Yoda database disappeared. Now let's check the Yoda web interface.
![alt text](Yoda_metadata_web.png "Title")

Yoda does also not allow us to add the metadata item again through the API. So we created inconsistent data.

In [ ]:
coll.metadata.add("Family_Name", "Staiger", "usr_6_s")

**Note, that only the metadata.json (which is rendered in the web interface) will be archived!!!**. The metadata in the Yoda database will be ignored in thos step.

## Changing Yoda metadata for workflows
One of the metadata items in the list above is actually not only metadata but also a trigger for certain actions:
`org_action_log ["1663576969", "submitted for vault", "c.staiger@uu.nl#nluu12p"] None`.
This metadata indicates that the data has been submitted to the vault and your data manager received a message.

![alt text](submitted_vault.png "Title")
What happens if we change this metadata?

In [ ]:
m_item = coll.metadata.items()[0]
print(m_item)

In [ ]:
coll.metadata.remove(m_item)

Since the collection is no longer markerd as "submitted", the data manager does not get the drop down menu's choice to accept or reject the collection for archiving any longer and the procedure is interrupted:
![alt text](resubmit.png "Title")

The same is true for the metadata tag that triggers the replication of your data to another data centre.

## Accession Control Lists
Yoda uses accession control lists (ACLs) to manage who can do what on the data.

In [28]:
objPath = "/nluu12p/home/research-test-christine/books/AdventuresSherlockHolmes.txt"
obj = session.data_objects.get(objPath)

In [29]:
[vars(p) for p in session.permissions.get(obj)]

/var/folders/7z/3klc4jw14w9brkxtlk_312gh0000gp/T/ipykernel_8083/3818099567.py:1: DeprecationWarning: Use of session_obj.permissions is deprecated in v1.1.6
  [vars(p) for p in session.permissions.get(obj)]


[{'access_name': 'read object',
  'path': '/nluu12p/home/research-test-christine/books/AdventuresSherlockHolmes.txt',
  'user_name': 'datamanager-its',
  'user_zone': 'nluu12p',
  'user_type': 'rodsgroup'},
 {'access_name': 'own',
  'path': '/nluu12p/home/research-test-christine/books/AdventuresSherlockHolmes.txt',
  'user_name': 'rods',
  'user_zone': 'nluu12p',
  'user_type': 'rodsadmin'}]

The data is owned by your research group,  `research-test-christine`, 3rd entry. This is set up by the Yoda group manager module. Let's see if we can retract `own` right and hide our data object from the group:

In [ ]:
from irods.access import iRODSAccess
acl = iRODSAccess('own', obj.path, 'research-test-christine', session.zone)

In [ ]:
session.permissions.set(acl)
[vars(p) for p in session.permissions.get(obj)]

*Yes*, we can ess up Yoda's group ACL structure.

## Working with data in the Vault
### Prerequisites
- An annotated collection with some files 
- Move the collection through the workflow in Yoda to the Vault (Front end)

### Experiment
- Check the ACLs and try to give your account `write` rights
- Get the archived metadata in the iCAT and change it

In [10]:
homeCollPath = '/' + session.zone + '/home/vault-test-christine'

In [11]:
coll = session.collections.get(homeCollPath)

In [17]:
data_coll = coll.subcollections[0]
assert data_coll.owner_name != session.username

In [31]:
from irods.exception import CAT_NO_ACCESS_PERMISSION
from irods.access import iRODSAccess
acl = iRODSAccess('own', data_coll.path, 'c.staiger@uu.nl', session.zone)
try:
    session.permissions.set(acl) # Should return CAT_NO_ACCESS_PERMISSION
    print("Set ACLs in Vault prohibited: Test failed")
except CAT_NO_ACCESS_PERMISSION:
    print("Set ACLs in Vault prohibited: Test passed")
except Exception:
    print("Fail")

Set ACLs in Vault prohibited: Test passed


/var/folders/7z/3klc4jw14w9brkxtlk_312gh0000gp/T/ipykernel_8083/440074154.py:5: DeprecationWarning: Use of session_obj.permissions is deprecated in v1.1.6
  session.permissions.set(acl) # Should return CAT_NO_ACCESS_PERMISSION


In [34]:
metadata = data_coll.metadata.items()[0]
data_coll.metadata.remove(metadata)


CAT_NO_ACCESS_PERMISSION: None

In [37]:
obj = data_coll.data_objects[0]
obj

<iRODSDataObject 21368138 yoda-metadata[1698656928].json>

In [38]:
obj.unlink()

CAT_NO_ACCESS_PERMISSION: None